In [4]:
import torch
import torch.nn as nn

In [2]:
# configuration of GPT-2 model
GPT_CONFIG_124M = {
    "vocab_size": 50_257, # number of tokens of BPE tokenizer
    "context_length": 1_024, # maximum number of input tokens the model can handle via positional embedding
    "emb_dim": 768, # embedding size, transforms each token into 768-dimensional vector
    "n_heads": 12, # count of attention heads in the multi.head attention mechanism
    "n_layers": 12, # count of transformer block in the model NOTE: (does it have to be equal to number of heads?)
    "drop_rate": 0.1,
    "qkv_bias": False
}

A placeholder GPT model architecture class

In [ ]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        
        self.tok_emb = nn.Embedding(num_embeddings=cfg["vocab_size"], embedding_dim=cfg["emb_dim"])
        self.pos_emb = nn.Embedding(num_embeddings=cfg["vocab_size"], embedding_dim=cfg["emb_dim"])
        self.drop_emb = nn.Dropout(p=cfg["drop_rate"])
        # Use a placeholder for TransformerBlock
        self.trf_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) 
              for _ in range(cfg["n_layers"])]
        )
        # Use a placeholder for LayerNorm
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        # final layer for prediction, it will output probability for each token
        self.out_head = nn.Linear(
           in_features=cfg["emb_dim"], out_features=["vocab_size"],
           bias=False 
        )
    
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(
            torch.arange()
        )


class DummyTransformerBlock(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, x):
        pass

class DummyLayerNorm(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def forward(self, x):
        oass

SyntaxError: invalid syntax (4199948351.py, line 4)